In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("enricogrimaldi/falls-vs-normal-activities")

print("Path to dataset files:", path)

100%|██████████| 1.24M/1.24M [00:00<00:00, 97.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/enricogrimaldi/falls-vs-normal-activities/versions/1


In [4]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
import os

In [6]:
print(os.listdir(path))

['acc_gyr.csv']


In [7]:
import pandas as pd

In [8]:
data_dir = os.path.join(path,'acc_gyr.csv')
df = pd.read_csv(data_dir)

In [23]:
df.head(100)

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
95,5.39,0.67,-7.16,-55.11,10.93,41.99,fall
96,5.50,0.68,-7.33,-53.16,17.40,36.19,fall
97,6.78,-0.03,-7.24,-60.18,16.60,39.12,fall
98,5.65,-0.11,-7.67,-60.55,15.20,43.27,fall


In [10]:
unique_dt = df['label'].value_counts()
print(unique_dt)

label
light    27600
walk     19200
fall     18401
sit      16000
step      6000
rfall     5599
lfall     4000
Name: count, dtype: int64


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96800 entries, 0 to 96799
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   xAcc    96800 non-null  float64
 1   yAcc    96800 non-null  float64
 2   zAcc    96800 non-null  float64
 3   xGyro   96800 non-null  float64
 4   yGyro   96800 non-null  float64
 5   zGyro   96800 non-null  float64
 6   label   96800 non-null  object 
dtypes: float64(6), object(1)
memory usage: 5.2+ MB


In [14]:
num_observations = 242
timesteps = 400
num_features = 6

# Simulated data (replace with actual dataset)
np.random.seed(42)
X = df.drop('label', axis=1).values.reshape(num_observations, timesteps, num_features)
y = df['label'].values[::timesteps] # Modified to select one label per observation

# Normalize accelerometer (cols 0-2) and gyroscope (cols 3-5)
def normalize_signals(X):
    X_norm = X.copy()
    for i in range(X.shape[0]):
        # Accelerometer
        acc = X[i, :, 0:3]
        acc_norm = acc / (np.linalg.norm(acc, axis=1, keepdims=True) + 1e-8)
        # Gyroscope
        gyr = X[i, :, 3:6]
        gyr_norm = gyr / (np.linalg.norm(gyr, axis=1, keepdims=True) + 1e-8)
        # Combine back
        X_norm[i, :, 0:3] = acc_norm
        X_norm[i, :, 3:6] = gyr_norm
    return X_norm

X = normalize_signals(X)

# Flatten each observation: 400*6 = 2400 features
X_flat = X.reshape(num_observations, timesteps * num_features)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [15]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform y_train and y_test
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Update y_train and y_test to their encoded versions
y_train = y_train_encoded
y_test = y_test_encoded

# Print the mapping of labels to integers
print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))
print("Example of encoded y_train:", y_train[:5])
print("Example of encoded y_test:", y_test[:5])

Label mapping: {'fall': np.int64(0), 'lfall': np.int64(1), 'light': np.int64(2), 'rfall': np.int64(3), 'sit': np.int64(4), 'step': np.int64(5), 'walk': np.int64(6)}
Example of encoded y_train: [4 0 5 4 2]
Example of encoded y_test: [3 3 5 0 2]


In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(timesteps*num_features,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Updated to reflect the actual number of classes
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     1,229,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,394,439 (5.32 MB)

 Trainable params: 1,394,439 (5.32 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=16
)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.3438 - loss: 2.0560 - val_accuracy: 0.7436 - val_loss: 0.7701
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7718 - loss: 0.8094 - val_accuracy: 0.9231 - val_loss: 0.4195
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8706 - loss: 0.3559 - val_accuracy: 0.7436 - val_loss: 0.6330
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9180 - loss: 0.4441 - val_accuracy: 0.7179 - val_loss: 0.6293
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8937 - loss: 0.2509 - val_accuracy: 0.7949 - val_loss: 0.5244
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9538 - loss: 0.1614 - val_accuracy: 0.7692 - val_loss: 0.5961
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9743 - loss: 0.0716 - val_accuracy: 0.7692 - val_loss: 0.7808
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9400 - loss: 0.2305 - val_accuracy: 0.8462 - v

In [ ]:
model.save("fall_detection_model.keras")

# TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("fall_detection_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model saved as H5 and TFLite!")

In [28]:
import numpy as np

label_map = {
    0: 'fall',
    1: 'lfall',
    2: 'light',
    3: 'rfall',
    4: 'sit',
    5: 'step',
    6: 'walk'
}

test_window = np.random.rand(400, 6).astype(np.float32)

test_input = test_window.reshape(1, 400 * 6)
pred = model.predict(test_input)
pred_class = np.argmax(pred, axis=1)[0]
pred_label = label_map[pred_class]

print("Raw prediction vector:", pred)
print("Predicted class index:", pred_class)
print("Predicted activity:", pred_label)



Exception ignored in: <function _xla_gc_callback at 0x7d7b9e15b100>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Raw prediction vector: [[1.5126072e-03 1.8061082e-03 3.1291202e-02 9.4214398e-01 1.4537379e-05
  2.2975475e-02 2.5605582e-04]]
Predicted class index: 3
Predicted activity: rfall
